# Exploration notebook
Phase 3

## IB API

In [1]:
import nest_asyncio
nest_asyncio.apply()

from src.execution.ib_connection import IBConnection, get_stock_contract

conn = IBConnection(port=7497)
conn.connect()

print(f"Connected: {conn.is_connected()}")
print(f"Accounts: {conn.ib.managedAccounts()}")

conn.ib.reqMarketDataType(3)

# Fetch a quote
aapl = get_stock_contract('AAPL')
conn.ib.qualifyContracts(aapl)

ticker = conn.ib.reqMktData(aapl)

import time
time.sleep(2)

print(f"\nSymbol: {aapl.symbol}")
print(f"Last price: {ticker.last}")
print(f"Bid: {ticker.bid}")
print(f"Ask: {ticker.ask}")

conn.ib.cancelMktData(aapl)
conn.disconnect()

Connected: True
Accounts: ['DUP678137']

Symbol: AAPL
Last price: nan
Bid: nan
Ask: nan


In [2]:
conn = IBConnection(port=7497)
conn.connect()

aapl = get_stock_contract('AAPL')
conn.ib.qualifyContracts(aapl)

# Fetch historical data
bars = conn.ib.reqHistoricalData(
    aapl,
    endDateTime='',  # Empty = now
    durationStr='5 D',  # Last 5 days
    barSizeSetting='1 hour',
    whatToShow='TRADES',
    useRTH=True  # Regular trading hours only
)

for bar in bars[-5:]:  # Print last 5 bars
    print(f"{bar.date} | O:{bar.open} H:{bar.high} L:{bar.low} C:{bar.close} V:{bar.volume}")

conn.disconnect()

2026-02-06 11:00:00-05:00 | O:278.83 H:279.78 L:278.49 C:279.09 V:3840227.0
2026-02-06 12:00:00-05:00 | O:279.05 H:279.47 L:277.9 C:278.3 V:2467273.0
2026-02-06 13:00:00-05:00 | O:278.3 H:278.69 L:276.92 C:277.48 V:2852821.0
2026-02-06 14:00:00-05:00 | O:277.49 H:278.65 L:276.99 C:278.58 V:2607749.0
2026-02-06 15:00:00-05:00 | O:278.56 H:279.01 L:277.56 C:278.01 V:5018576.0


In [1]:
import nest_asyncio
nest_asyncio.apply()

from src.execution.ib_fetcher import fetch_historical_data, fetch_multiple_symbols

# Single stock
df = fetch_historical_data('AAPL', duration='1 M', bar_size='1 day')
print(df.tail())
print(df.shape)

# Several stocks
df = fetch_multiple_symbols(['AAPL','MSFT','GOOGL','NVDA','AMZN'], duration='1 M', bar_size='1 day')
print(df.tail())
print(df.shape)

              Open    High     Low   Close      Volume ticker
date                                                         
2026-02-02  260.02  270.49  259.20  270.01  39664024.0   AAPL
2026-02-03  269.20  271.88  267.61  269.48  33472513.0   AAPL
2026-02-04  272.29  278.95  272.28  276.49  48895823.0   AAPL
2026-02-05  278.13  279.50  273.23  275.91  28324221.0   AAPL
2026-02-06  277.18  280.91  276.92  278.12  28792933.0   AAPL
(21, 6)
              Open    High     Low   Close       Volume ticker
date                                                          
2026-02-02  238.21  245.63  238.17  242.96   20838117.0   AMZN
2026-02-03  245.10  246.35  235.45  238.62   31007967.0   AMZN
2026-02-04  238.86  238.86  231.82  232.99   28074250.0   AMZN
2026-02-05  224.91  226.31  220.38  222.69   48560277.0   AMZN
2026-02-06  202.69  211.44  200.31  210.32  111610818.0   AMZN
(105, 6)


## FRED API

In [2]:
from fredapi import Fred
from dotenv import load_dotenv
import os
import pandas as pd

load_dotenv()
fred = Fred(api_key=os.getenv('FRED_API_KEY'))

# Test: fetch US 10-year Treasury yield
us10y = fred.get_series('GS10')
print(us10y.tail())

2025-09-01    4.12
2025-10-01    4.06
2025-11-01    4.09
2025-12-01    4.14
2026-01-01    4.21
dtype: float64


In [3]:
from fredapi import Fred
from dotenv import load_dotenv
import os
import pandas as pd
import time
import logging

load_dotenv()
fred = Fred(api_key=os.getenv('FRED_API_KEY'))

# Fetch a single series
raw = fred.get_series('GS10', observation_start='2020-01-01')
print(type(raw))
print(raw.head())

<class 'pandas.Series'>
2020-01-01    1.76
2020-02-01    1.50
2020-03-01    0.87
2020-04-01    0.66
2020-05-01    0.67
dtype: float64
